![Cabec%CC%A7alho_notebook.png](cabecalho_notebook.png)

# PCA - Tarefa 01: *HAR* com PCA

Vamos trabalhar com a base da demonstração feita em aula, mas vamos explorar um pouco melhor como é o desempenho da árvore variando o número de componentes principais.

In [9]:
import pandas as pd

from sklearn.tree import DecisionTreeClassifier

from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

X_train = pd.read_csv('./X_train.csv')
X_test = pd.read_csv('./X_test.csv')
y_train = pd.read_csv('./y_train.txt', header=None, delim_whitespace=True)
y_test = pd.read_csv('./y_test.txt', header=None, delim_whitespace=True)

In [10]:
X_train.set_index(['subject', X_train.index], inplace=True)
X_test.set_index(['subject', X_test.index], inplace=True)

## Árvore de decisão

Rode uma árvore de decisão com todas as variáveis, utilizando o ```ccp_alpha=0.001```. Avalie a acurácia nas bases de treinamento e teste. Avalie o tempo de processamento.

In [11]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train)

In [14]:
%%time

clf = DecisionTreeClassifier(random_state=1234, ccp_alpha=0.001).fit(X_train, y_train)

# vendo o score da arvore
acc_train = clf.score(X_train, y_train)
acc_valid = clf.score(X_valid, y_valid)
acc_teste = clf.score(X_test, y_test)

print(f'Acurácia da melhor árvore na base de treino:    {acc_train*100:.1f}')
print(f'Acurácia da melhor árvore na base de validação: {acc_teste*100:.1f}')
print(f'Acurácia da melhor árvore na base de teste:     {acc_teste*100:.1f}')


Acurácia da melhor árvore na base de treino:    97.8
Acurácia da melhor árvore na base de validação: 85.2
Acurácia da melhor árvore na base de teste:     85.2
CPU times: total: 2.28 s
Wall time: 2.28 s


## Árvore com PCA

Faça uma análise de componemtes principais das variáveis originais. Utilize apenas uma componente. Faça uma árvore de decisão com esta componente como variável explicativa.

- Avalie a acurácia nas bases de treinamento e teste
- Avalie o tempo de processamento

In [21]:
%%time
# numero de componentes desejadas
n=1
#avaliação das componenetes
prcomp = PCA(n_components=n).fit(X_train)

pc_treino = prcomp.transform(X_train)
pc_valida = prcomp.transform(X_valid)
pc_teste  = prcomp.transform(X_test)


CPU times: total: 734 ms
Wall time: 425 ms


In [22]:
# separando em df- para treino teste e validação
colunas = ['cp'+str(x+1) for x in list(range(n))]

pc_train = pd.DataFrame(pc_treino[:,:n], columns = colunas)
pc_valid = pd.DataFrame(pc_valida[:,:n], columns = colunas)
pc_test  = pd.DataFrame( pc_teste[:,:n], columns = colunas)

pc_train.head()

,cp1
0,75.570739
1,-2516.426289
2,-2986.429176
3,3004.572525
4,1991.573079


In [23]:
%%time

clf_pca = DecisionTreeClassifier(random_state=1234, ccp_alpha=0.001).fit(pc_train, y_train)

# vendo o score da arvore
acc_train = clf_pca.score(pc_train, y_train)
acc_valid = clf_pca.score(pc_valid, y_valid)
acc_teste = clf_pca.score(pc_test, y_test)

print(f'Acurácia da melhor árvore na base de treino:    {acc_train*100:.1f}')
print(f'Acurácia da melhor árvore na base de validação: {acc_teste*100:.1f}')
print(f'Acurácia da melhor árvore na base de teste:     {acc_teste*100:.1f}')

Acurácia da melhor árvore na base de treino:    98.6
Acurácia da melhor árvore na base de validação: 13.5
Acurácia da melhor árvore na base de teste:     13.5
CPU times: total: 62.5 ms
Wall time: 67.1 ms


## Testando o número de componentes

Com base no código acima, teste a árvore de classificação com pelo menos as seguintes possibilidades de quantidades de componentes: ```[1, 2, 5, 10, 50]```. Avalie para cada uma delas:

- Acurácia nas bases de treino e teste
- Tempo de processamento


In [35]:
from time import time

In [41]:
 %%time
#numero de componenetes
n = [1, 2, 5, 10, 50]
#avaliação das componenetes
for i in n:
    start_time = time()
    prcomp = PCA(n_components=i).fit(X_train)
        
    pc_treino = prcomp.transform(X_train)
    pc_valida = prcomp.transform(X_valid)
    pc_teste  = prcomp.transform(X_test)
    
    colunas = ['cp'+str(i+1) for x in list(range(i))]

    pc_train = pd.DataFrame(pc_treino[:,:i], columns = colunas)
    pc_valid = pd.DataFrame(pc_valida[:,:i], columns = colunas)
    pc_test  = pd.DataFrame( pc_teste[:,:i], columns = colunas) 
    
    #treinando a arvore
    clf_pca = DecisionTreeClassifier(random_state=1234, ccp_alpha=0.001).fit(pc_train, y_train)
    # vendo o score da arvore
    acc_train = clf_pca.score(pc_train, y_train)
    acc_valid = clf_pca.score(pc_valid, y_valid)
    acc_teste = clf_pca.score(pc_test, y_test)
    end_time = time()
    total_time = end_time - start_time
    
    print(f'Resultados na arvore com compoenetes:    {i}')
    print(f"O tempo total de execução foi de {total_time:.2f} segundos.")
    print(f'Acurácia da melhor árvore na base de treino:    {acc_train*100:.1f}')
    print(f'Acurácia da melhor árvore na base de validação: {acc_teste*100:.1f}')
    print(f'Acurácia da melhor árvore na base de teste:     {acc_teste*100:.1f}')

Resultados na arvore com compoenetes:    1
O tempo total de execução foi de 0.50 segundos.
Acurácia da melhor árvore na base de treino:    98.6
Acurácia da melhor árvore na base de validação: 13.5
Acurácia da melhor árvore na base de teste:     13.5
Resultados na arvore com compoenetes:    2
O tempo total de execução foi de 0.51 segundos.
Acurácia da melhor árvore na base de treino:    85.6
Acurácia da melhor árvore na base de validação: 37.6
Acurácia da melhor árvore na base de teste:     37.6
Resultados na arvore com compoenetes:    5
O tempo total de execução foi de 0.65 segundos.
Acurácia da melhor árvore na base de treino:    88.1
Acurácia da melhor árvore na base de validação: 72.9
Acurácia da melhor árvore na base de teste:     72.9
Resultados na arvore com compoenetes:    10
O tempo total de execução foi de 0.81 segundos.
Acurácia da melhor árvore na base de treino:    90.2
Acurácia da melhor árvore na base de validação: 80.7
Acurácia da melhor árvore na base de teste:     80.7

## Conclua

- O que aconteceu com a acurácia?
- O que aconteceu com o tempo de processamento?

A acurácia aumenta até um limite proximo ao usar o df todo, porém isso custou mais processamento, tendo uma queda de performance. 